In [1]:
import pandas as pd
import xml.etree.ElementTree as et
import os
import numpy as np
import matplotlib.pyplot as plt
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.corpus import stopwords
max_features=5000
max_sequence_length=500

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')
base_dir='/content/drive/My Drive'
os.chdir(base_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 1.1 - Importing Web Of Knowledge papers

In [3]:
wos = pd.read_html('webofscience_PDT_01.html')#, sep=";",error_bad_lines=False)
print('First Web of Knowledge database number of papers:',len(wos))
AB=[] # Abstract
PM=[] # PUBMED ID
PY=[] # Year Published
SC=[] # Research Area
KW=[] # Paper Keywords
PT=[] # Publication Type - Journal;Book;Series;Patent
TI=[] # Document Title
DI=[] # DOI
for ii in range(len(wos)):
  aux2=np.where(wos[ii][0]=="AB")
  if len(aux2[0])==0:
    AB.append('None')
  else:
    idx=aux2[0][0]
    AB.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="PM")
  if len(aux2[0])==0:
    PM.append('None')
  else:
    idx=aux2[0][0]
    PM.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="PY")
  if len(aux2[0])==0:
    PY.append('None')
  else:
    idx=aux2[0][0]
    PY.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="SC")
  if len(aux2[0])==0:
    SC.append('None')
  else:
    idx=aux2[0][0]
    SC.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="DE")
  if len(aux2[0])==0:
    KW.append('None')
  else:
    idx=aux2[0][0]
    KW.append(wos[ii][1][idx])    
  aux2=np.where(wos[ii][0]=="PT")
  if len(aux2[0])==0:
    PT.append('None')
  else:
    idx=aux2[0][0]
    PT.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="TI")
  if len(aux2[0])==0:
    TI.append('None')
  else:
    idx=aux2[0][0]
    TI.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="DI")
  if len(aux2[0])==0:
    DI.append('None')
  else:
    idx=aux2[0][0]
    DI.append(wos[ii][1][idx])

df_wos1=pd.DataFrame(PM)
df_wos1.columns=['pmid']
df_wos1['doi_wos']=DI
df_wos1['title_wos']=TI
df_wos1['abstract_wos']=AB
df_wos1['publication_year']=PY
df_wos1['research_area']=SC
df_wos1['keywords']=KW
df_wos1['publication_type']=PT
df_wos1 = df_wos1.drop(df_wos1[df_wos1.title_wos=='None'].iloc[:].index)

First Web of Knowledge database number of papers: 855


In [4]:
wos = pd.read_html('webofscience_PDT_02.html')#, sep=";",error_bad_lines=False)
print('Second Web of Knowledge database number of papers:',len(wos))
AB=[] # Abstract
PM=[] # PUBMED ID
PY=[] # Year Published
SC=[] # Research Area
KW=[] # Paper Keywords
PT=[] # Publication Type - Journal;Book;Series;Patent
TI=[] # Document Title
DI=[] # DOI
for ii in range(len(wos)):
  aux2=np.where(wos[ii][0]=="AB")
  if len(aux2[0])==0:
    AB.append('None')
  else:
    idx=aux2[0][0]
    AB.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="PM")
  if len(aux2[0])==0:
    PM.append('None')
  else:
    idx=aux2[0][0]
    PM.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="PY")
  if len(aux2[0])==0:
    PY.append('None')
  else:
    idx=aux2[0][0]
    PY.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="SC")
  if len(aux2[0])==0:
    SC.append('None')
  else:
    idx=aux2[0][0]
    SC.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="DE")
  if len(aux2[0])==0:
    KW.append('None')
  else:
    idx=aux2[0][0]
    KW.append(wos[ii][1][idx])    
  aux2=np.where(wos[ii][0]=="PT")
  if len(aux2[0])==0:
    PT.append('None')
  else:
    idx=aux2[0][0]
    PT.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="TI")
  if len(aux2[0])==0:
    TI.append('None')
  else:
    idx=aux2[0][0]
    TI.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="DI")
  if len(aux2[0])==0:
    DI.append('None')
  else:
    idx=aux2[0][0]
    DI.append(wos[ii][1][idx])

df_wos2=pd.DataFrame(PM)
df_wos2.columns=['pmid']
df_wos2['doi_wos']=DI
df_wos2['title_wos']=TI
df_wos2['abstract_wos']=AB
df_wos2['publication_year']=PY
df_wos2['research_area']=SC
df_wos2['keywords']=KW
df_wos2['publication_type']=PT
df_wos2 = df_wos2.drop(df_wos2[df_wos2.title_wos=='None'].iloc[:].index)


Second Web of Knowledge database number of papers: 878


In [5]:
wos = pd.read_html('webofscience_PDT_03.html')#, sep=";",error_bad_lines=False)
print('Third Web of Knowledge database number of papers:',len(wos))
AB=[] # Abstract
PM=[] # PUBMED ID
PY=[] # Year Published
SC=[] # Research Area
KW=[] # Paper Keywords
PT=[] # Publication Type - Journal;Book;Series;Patent
TI=[] # Document Title
DI=[] # DOI
for ii in range(len(wos)):
  aux2=np.where(wos[ii][0]=="AB")
  if len(aux2[0])==0:
    AB.append('None')
  else:
    idx=aux2[0][0]
    AB.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="PM")
  if len(aux2[0])==0:
    PM.append('None')
  else:
    idx=aux2[0][0]
    PM.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="PY")
  if len(aux2[0])==0:
    PY.append('None')
  else:
    idx=aux2[0][0]
    PY.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="SC")
  if len(aux2[0])==0:
    SC.append('None')
  else:
    idx=aux2[0][0]
    SC.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="DE")
  if len(aux2[0])==0:
    KW.append('None')
  else:
    idx=aux2[0][0]
    KW.append(wos[ii][1][idx])    
  aux2=np.where(wos[ii][0]=="PT")
  if len(aux2[0])==0:
    PT.append('None')
  else:
    idx=aux2[0][0]
    PT.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="TI")
  if len(aux2[0])==0:
    TI.append('None')
  else:
    idx=aux2[0][0]
    TI.append(wos[ii][1][idx])
  aux2=np.where(wos[ii][0]=="DI")
  if len(aux2[0])==0:
    DI.append('None')
  else:
    idx=aux2[0][0]
    DI.append(wos[ii][1][idx])
    
    
df_wos3=pd.DataFrame(PM)
df_wos3.columns=['pmid']
df_wos3['doi_wos']=DI
df_wos3['title_wos']=TI
df_wos3['abstract_wos']=AB
df_wos3['publication_year']=PY
df_wos3['research_area']=SC
df_wos3['keywords']=KW
df_wos3['publication_type']=PT
df_wos3 = df_wos3.drop(df_wos3[df_wos3.title_wos=='None'].iloc[:].index)

Third Web of Knowledge database number of papers: 420


In [6]:
df_wos=pd.concat([df_wos1,df_wos2,df_wos3],ignore_index=True)
del df_wos1, df_wos2, df_wos3
df_wos.head()

,pmid,doi_wos,title_wos,abstract_wos,publication_year,research_area,keywords,publication_type
0,31500949,10.1016/j.clon.2019.08.005,Management of Non-melanoma Skin Cancer in Tran...,Transplant recipients have a significantly hig...,2019,Oncology,Dermatology; immunosuppression; non-melanoma s...,J
1,31543301,10.1016/j.clon.2019.08.010,Electrochemotherapy and Ablative Therapies in ...,Although surgery and radiotherapy remain the m...,2019,Oncology,Ablative; electrochemotherapy; keratinocyte tu...,J
2,31355928,"10.1002/cncr.32371,","Surgical excision, Mohs micrographic surgery, ...",Background The objective of this study was to ...,2019,Oncology,basal cell carcinoma; brachytherapy; external-...,J
3,31466584,10.1016/j.det.2019.05.003,Nonsurgical Treatments for Nonmelanoma Skin Ca...,Although surgical intervention remains the sta...,2019,Dermatology,Nonmelanoma skin cancer (NMSC); Basal cell car...,J
4,31555659,10.3389/fsurg.2019.00045,Delta-Aminolevulinic Acid-Mediated Photodiagno...,Fluorescence imaging is an emerging clinical t...,2019,Surgery,fluorescence; 5ALA; protoporphyrin IX; surgery...,J


# 1.2 - Importing and mounting dataframe from pubmed XML

In [9]:
pubmedXML=et.parse('pubmed_result_PDT_semfiltro.xml')
print('IMPORTING PUBMED DATABASE')
print('Number of papers in PubMed database:',len(pubmedXML.getroot()))

IMPORTING PUBMED DATABASE
Number of papers in PubMed database: 719


In [12]:
DOI=[]
count=0
count2=0
count3=0
for ii in range(len(pubmedXML.getroot())):
  a3=pubmedXML.getroot()[ii][0][3].findall('ELocationID')
  if len(a3)==0:
    count=count+1
    DOI.append('None')
  else:
    for jj in range(len(a3)):
      info=a3[jj]
      if info.items()[0][1]=='doi':
        DOI.append(info.text)
        break
      elif info.items()[0][1]=='pii':
        count2=count2+1
        DOI.append('None')
        break
print('Count of papers without ELocationID:',count)
print('Count of papers without DOI with pii:',count2)

Count of papers without ELocationID: 369
Count of papers without DOI with pii: 46


In [0]:
pmid=[]
count=0
for ii in range(len(pubmedXML.getroot())):
  a3=pubmedXML.getroot()[ii][0][0]
  if a3 ==None:
    pmid.append('None')
  else:
    pmid.append(a3.text)

In [0]:
Title=[]
count=0
for ii in range(len(pubmedXML.getroot())):
  a3=pubmedXML.getroot()[ii][0][3].find('ArticleTitle')
  if a3 ==None:
    Title.append('None')
  else:
    Title.append(a3.text)

In [0]:
Abs=[]
count=0
for ii in range(len(pubmedXML.getroot())):
  a3=pubmedXML.getroot()[ii][0][3].find('Abstract')
  if a3 ==None:
    Abs.append('None')
  else:
    text=''
    for jj in range(len(a3.getchildren())):
      abstract=a3.getchildren()
      if abstract==None:
        Abs.append('None')
      else:
        if abstract[jj].text==None:
          text='None'
        else:
          text=text+abstract[jj].text
    Abs.append(text)


Papers without Title information are dropped out

In [16]:
df_pubmed=pd.DataFrame(Title)
df_pubmed.columns=['title_pubmed']
df_pubmed['abstract_pubmed']=Abs
df_pubmed['pmid']=pmid
df_pubmed['doi_pubmed']=DOI
df_pubmed=df_pubmed.drop(df_pubmed[df_pubmed.title_pubmed=='None'].iloc[:].index)
df_pubmed=df_pubmed.reset_index()
df_pubmed=df_pubmed.drop('index',axis=1)
df_pubmed.head()

,title_pubmed,abstract_pubmed,pmid,doi_pubmed
0,Skin Cancer: Squamous and Basal Cell Carcinomas.,Approximately 1 in 5 Americans will develop sk...,31188548,None
1,A systematic review on the use of cryotherapy ...,Cryotherapy is a commonly discussed method for...,30695972,None
2,Histologic subtype of treatment failures after...,There have been concerns that recurrences afte...,30582993,None
3,Curettage and multiple needle puncture-assiste...,The use of skin puncture techniques may facili...,30530413,10.1684/ejd.2018.3445
4,Shedding light on photodynamic therapy for bas...,None,30508235,10.1111/bjd.17187


# 1.3 - Join Pubmed and WebofScience Dataframes

if join=1 
Papers databases PUBMED and WebOfKnowledge will be joined


if join =0, the clustering will be performed in each database

In [0]:
join=1

In [18]:
if join:
  print('Number of WebOfKnowledge papers:',df_wos.shape)
  print('Number of PubMed papers:',df_pubmed.shape)
  df_pubmed2=df_pubmed

Number of WebOfKnowledge papers: (1274, 8)
Number of PubMed papers: (680, 4)


In [19]:
if join:
  result=pd.merge(df_pubmed,
                 df_wos,
                 left_on='pmid',
                 right_on='pmid',
                 how='outer')
  print('Number of total papers:',result.shape)
  result.sample(10)

Number of total papers: (1452, 11)


In [0]:
if join:
  idx=np.where(pd.isnull(result.doi_wos))[0][:]
  result['final_doi']=result.doi_wos
  for ii in range(len(idx)):
    result.final_doi[idx[ii]]=result.doi_pubmed[idx[ii]]


  idx=np.where(pd.isnull(result.title_wos))[0][:]
  result['final_title']=result.title_wos
  for ii in range(len(idx)):
    result.final_title[idx[ii]]=result.title_pubmed[idx[ii]]


  idx=np.where(pd.isnull(result.abstract_pubmed))[0][:]
  result['final_abstract']=result.abstract_pubmed
  for ii in range(len(idx)):
    result.final_abstract[idx[ii]]=result.abstract_wos[idx[ii]]



# 1.4 - Prepare cleaning function

In [0]:
def clean_str(string):
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)

    cleanr = re.compile('<.*?>')

    string = re.sub(r'\d+', '', string)
    string = re.sub(cleanr, '', string)
    string = re.sub("'", '', string)
    string = re.sub(r'\W+', ' ', string)
    string = string.replace('_', '')


    return string.strip().lower()

# 2 Clustering Papers Part

In [23]:
if join:
  result['clean_abstract']=result['final_abstract'].apply(lambda x: clean_str(x))
  print(result.head())
else:
  df_pubmed['clean_abstract'] = df_pubmed['abstract_pubmed'].apply(lambda x: clean_str(x))
  df_wos['clean_abstract'] = df_wos['abstract_wos'].apply(lambda x: clean_str(x))
  print(df_pubmed.head())
  print('***')
  print(df_wos.head())

                                        title_pubmed  ...                                     clean_abstract
0   Skin Cancer: Squamous and Basal Cell Carcinomas.  ...  approximately in americans will develop skin c...
1  A systematic review on the use of cryotherapy ...  ...  cryotherapy is a commonly discussed method for...
2  Histologic subtype of treatment failures after...  ...  there have been concerns that recurrences afte...
3  Curettage and multiple needle puncture-assiste...  ...  the use of skin puncture techniques may facili...
4  Shedding light on photodynamic therapy for bas...  ...                                               none

[5 rows x 15 columns]


In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=False)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=200000,
                     max_df=0.9,
                     min_df=0.2,
                     stop_words='english',
                     ngram_range=(1,3))

def normalize_v2(x): 
  normalized = []
  words = nltk.word_tokenize(x)
  normalized.append(' '.join([stemmer.stem(word) for word in words if re.match('[a-zA-Z]+', word)]))
  return normalized


if join:
  text = []
  for row in result['clean_abstract'].values:
    text.append(normalize_v2(row)[0])
  result['clean_abstract']=text
else:
  text = []
  for row in df_pubmed['clean_abstract'].values:
    text.append(normalize_v2(row)[0])
  df_pubmed['clean_abstract']=text

  text = []
  for row in df_wos['clean_abstract'].values:
    text.append(normalize_v2(row)[0])
  df_wos['clean_abstract']=text
  


In [0]:
if join:
  tfidf_matrix = tfidf.fit_transform([x for x in result['clean_abstract']])
else:
  tfidf_matrix_pubmed = tfidf.fit_transform([x for x in df_pubmed['clean_abstract']])
  tfidf_matrix_wos = tfidf.fit_transform([x for x in df_wos['clean_abstract']])

In [26]:
if join:
  print('Merged PubMed and WoK TF-IDF matrix shape:',tfidf_matrix.get_shape())
else:
  print('PubMed TF-IDF matrix shape:',tfidf_matrix_pubmed.get_shape())
  print('WoK TF-IDF matrix shape:',tfidf_matrix_wos.get_shape())


Merged PubMed and WoK TF-IDF matrix shape: (1452, 47)


In [0]:
from sklearn.metrics.pairwise import cosine_similarity 
if join:
  similarity_distance = 1 - cosine_similarity(tfidf_matrix)
else:
  similarity_distance_pubmed = 1 - cosine_similarity(tfidf_matrix_pubmed)
  similarity_distance_wos = 1 - cosine_similarity(tfidf_matrix_wos)
  

In [28]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
if join:
  mergings = linkage(similarity_distance, method='complete')
  dendrogram_ = dendrogram(mergings,
                 labels=[x for x in result["final_title"]],
                 leaf_rotation=90,
                 leaf_font_size=16,
                )

  fig = plt.gcf()
  _ = [lbl.set_color('r') for lbl in plt.gca().get_xmajorticklabels()]
  fig.set_size_inches(108, 21)

  plt.show()
else:
  mergings = linkage(similarity_distance_pubmed, method='complete')
  dendrogram_ = dendrogram(mergings,
                 labels=[x for x in df_pubmed["title_pubmed"]],
                 leaf_rotation=90,
                 leaf_font_size=16,
                )

  fig = plt.gcf()
  _ = [lbl.set_color('r') for lbl in plt.gca().get_xmajorticklabels()]
  fig.set_size_inches(108, 21)

  plt.show()

  mergings = linkage(similarity_distance_wos, method='complete')
  dendrogram_ = dendrogram(mergings,
                 labels=[x for x in df_wos["title_wos"]],
                 leaf_rotation=90,
                 leaf_font_size=16,
                )

  fig = plt.gcf()
  _ = [lbl.set_color('r') for lbl in plt.gca().get_xmajorticklabels()]
  fig.set_size_inches(108, 21)

  plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
if join:
  def find_similar(title):
    index = result[result['final_title'] == title].index[0]
    vector = similarity_distance[index, :]
    most_similar = result.iloc[np.argsort(vector)[1:20], 0].values
    return most_similar
else:
  def find_similar_pubmed(title):
    index = df_pubmed[df_pubmed['title_pubmed'] == title].index[0]
    vector = similarity_distance_pubmed[index, :]
    most_similar = df_pubmed.iloc[np.argsort(vector)[1:20], 0].values
    return most_similar
  def find_similar_wos(title):
    index = df_wos[df_wos['title_wos'] == title].index[0]
    vector = similarity_distance_wos[index, :]
    most_similar = df_wos.iloc[np.argsort(vector)[1:20], 2].values
    return most_similar

In [30]:
Title='meta-analysis'
if join:
  index_=result['final_title'].str.contains(Title)
  print(result.final_title[index_].values[:])
  print('Merged final indexes order:',np.where(index_)[0])

else:
  index_pubmed=df_pubmed['title_pubmed'].str.contains(Title)
  print('Pubmed Titles:\n',df_pubmed.title_pubmed[index_pubmed].values[:])
  print('Pubmed indexes order:',np.where(index_pubmed)[0])
  
  index_wos=df_wos['title_wos'].str.contains(Title)
  print('\nWeb of Science Titles:\n',df_wos.title_wos[index_wos].values[:])
  print('Web of Science indexes order:',np.where(index_wos)[0])

['Photodynamic therapy versus surgical excision to basal cell carcinoma:  meta-analysis'
 'Photodynamic therapy in the treatment of basal cell carcinoma: a  systematic review and meta-analysis'
 'Overall treatment success after treatment of primary superficial basal  cell carcinoma: a systematic review and meta-analysis of randomized and  nonrandomized trials'
 'Surgical excision, Mohs micrographic surgery, external-beam  radiotherapy, or brachytherapy for indolent skin cancer: An  international meta-analysis of 58 studies with 21,000 patients'
 'Surgical excision, Mohs micrographic surgery, external-beam  radiotherapy, or brachytherapy for indolent skin cancer: An  international meta-analysis of 58 studies with 21,000 patients'
 'Efficacy and safety of imiquimod 5% cream for basal cell carcinoma: a  meta-analysis of randomized controlled trial'
 'Conventional and combination topical photodynamic therapy for basal cell  carcinoma: systematic review and meta-analysis']
Merged final inde

In [31]:
if join:
  index_=81
  print('Base title:',result.final_title[index_]) # prints
  abst=(result.final_abstract[index_])
  if abst =='None':
    print('No Abstract available for this paper')
  else: 
    print('\nMost similar title:\n',find_similar(result.final_title[index_])) # prints  
else:
  index_pubmed=343
  print('Pubmed Base title:',df_pubmed.title_pubmed[index_pubmed]) # prints
  abst=(df_pubmed.abstract_pubmed[index_pubmed])
  if abst =='None':
    print('No Abstract available for this paper')
  else: 
    print('\n Pubmed database - Most similar title:\n',find_similar_pubmed(df_pubmed.title_pubmed[index_pubmed])) # prints 
  
  print('\n _____________________________________________________________________________________________________________________________________________________________________________________________________ \n')
  
  index_wos=740
  print('Web of ScienceBase title:',df_wos.title_wos[index_wos]) # prints 
  abst=(df_wos.abstract_wos[index_wos])
  if abst =='None':
    print('No Abstract available for this paper')
  else: 
    print('\n Web of Science database - Most similar title:\n',find_similar_wos(df_wos.title_wos[index_wos])) # prints 

Base title: Photodynamic therapy versus surgical excision to basal cell carcinoma:  meta-analysis

Most similar title:
 ['Photodynamic Therapy Followed by Mohs Micrographic Surgery Compared to Mohs Micrographic Surgery Alone for the Treatment of Basal Cell Carcinoma: Results of a Pilot Single-Blinded Randomised Controlled Trial.'
 'PDT in squamous cell carcinoma of the skin.'
 'Topical photodynamic therapy significantly reduces epidermal Langerhans cells during clinical treatment of basal cell carcinoma.'
 'Comprehensive analysis of alterations in the miRNome in response to photodynamic treatment.'
 'Pre-treatment deep curettage can significantly reduce tumour thickness in thick Basal cell carcinoma while maintaining a favourable cosmetic outcome when used in combination with topical photodynamic therapy.'
 'Daylight-mediated photodynamic therapy in Spain: advantages and disadvantages.'
 'Therapeutic effect of Imiquimod enhanced ALA-PDT on cutaneous squamous cell carcinoma.'
 'Photodyn